In [1]:
import pandas as pd
import requests
import json
import time
from git import Repo
import re
from pathlib import Path
import git
from projects import project_list

In [2]:
fe = open('file_extensions.json', 'r')
file_extensions = json.load(fe)
types = list(file_extensions.keys())

In [3]:
def ignoreTheFormate(string_code):
    int1 = re.sub('/\/\/(.*)|\/\*(\*(?!\/)|[^*])*\*\//g', '', string_code) #removing single and multiline java comments
    int2 = re.sub('/([\\\]+<>=*(){}[,;-])/g', "\n$1\n", int1) 
    int3 = re.sub('/\s(\s*)/g', "\n", int2) #removing white spaces
    return int3.strip()


In [4]:
def getType(extension):
    
    for i in range(len(types)):
        if extension in file_extensions[types[i]]:
            return types[i]

    if (extension == ''):
        file_name = path.basename(file)
        if (file_name.indexOf('.git') == 0 |
            file_name.indexOf('.bzr') == 0 |
            file_name.indexOf('.svn') == 0 |
            file_name.indexOf('.cvs') == 0):
            return 'meta'
    
    return "unknown"


In [6]:
discard_list = []
for p in project_list[:3]:
    tr_data = pd.read_csv('data/25_1_travis_data/' + p)
    
    branch_type = tr_data['git_branch'].tolist()
    if 'master' in branch_type:
        data = tr_data[ tr_data['git_branch'] == 'master']
    else:
        data = tr_data[ tr_data['git_branch'] == 'trunk']
        
    if len(data) == 0:
        print('Could not continue for {}'.format(p))
        discard_list.append(p)
        continue
    
    repo_name = p.split('/')[0]
    clone_dir = 'project_clones/' + repo_name
    
    repo = Repo(clone_dir)
    verdict_list = [] #final decision

    for x in range(len(tr_data)):
        row = tr_data.iloc[x]
        commit_id = row['git_trigger_commit']
        print(commit_id)
        
        try:
            commit = repo.commit(commit_id)
        except:
            print('omitting {}'.format(commit_id))
            verdict_list.append(0)
            continue

        file_flag_list = []
        code_flag_list = []
        format_flag_list = []

        verdict = 1
    
        for file_name in commit.stats.files:

            extension = '.' + file_name.split('.')[-1]
            t = getType(extension)

            if (t == 'meta') | (t == 'media') | (t == 'doc'):
                file_flag_list.append(1)
                verdict = verdict & 1
                #we have found non-source code changes
                #we have found meta file changes

                #we have found that this file can be skipped 

            elif extension == '.java':
                try:
                    new_version = ignoreTheFormate(repo.git.show('{}:{}'.format(commit_id, file_name)))
                    old_version = ignoreTheFormate(repo.git.show('{}:{}'.format(commit_id + '~', file_name)))

                    if old_version != new_version:
                        #we have found code changes
                        #we have found that this file cannot be skipped
                        code_flag_list.append(0)
                        verdict = verdict & 0
                    else:
                        #we have found file formatting or comment changes
                        #we have found that this file can be skipped 
                        format_flag_list.append(1)
                        verdict = verdict & 1
                except:
                    #file added or deleted
                    print('File {} added or deleted'.format(file_name))
                    verdict = verdict & 0

            else:
                #out of skip rules scope
                verdict = verdict & 0
                continue

        verdict_list.append(verdict)
    
    tr_data['verdict'] = verdict_list
    p_name = p.split('/')[1]
    tr_data.to_csv( 'ci_skip_data/extracted_project_travis/'+ p_name )
    
    print(verdict_list.count(1), len(verdict_list), 100*verdict_list.count(1)/len(verdict_list))

fede50e0c40a803b87603312174f31282e6aeb06
0ab1474b798dd3a502ecf8b9156ee874ea84eec5
File src/main/java/org/junit/lambda/prototype01/ClassExecutionContext.java added or deleted
File src/main/java/org/junit/lambda/prototype01/InnerClassExecutionContext.java added or deleted
File src/main/java/org/junit/lambda/prototype01/MethodExecutionContext.java added or deleted
File src/main/java/org/junit/lambda/prototype01/MyTestExecutionListener.java added or deleted
File src/main/java/org/junit/lambda/prototype01/TestExecutionContext.java added or deleted
File src/main/java/org/junit/lambda/prototype01/TestExecutionListener.java added or deleted
File src/main/java/org/junit/lambda/prototype01/TestExecutionListener2.java added or deleted
ee1ef35b795f147cbaee73b2f36cba00eb72bde6
d9653cc3534dbe8945c9fbd0eb0f68f78e561585
9dc1132371f1b737133a891691534e5ef5bd759c
File junit-lambda-core/src/main/java/org/junit/lambda/core/Assertions.java added or deleted
4723b3e838ff1b947303e3d51fc0077deee7d602
334e910579

a52d04ff1b9a42899242b1b9cc7a8147bb2f3776
File junit-engine-api/src/main/java/org/junit/gen5/engine/TestListenerRegistry2.java added or deleted
b96027475481f05a26bb079c13671744a3b0921a
8af7d2c4a2dfa5695ed76c4ca01e05596006432c
File junit5-api/src/main/java/org/junit/gen5/api/Before.java added or deleted
a5a85a020486f7a29922aa6cdab70f1a0dc51118
5c52d3c13b9e4ca0e378f550a5bb1e691b4fb76e
7c81bd9f21cb41b5a2affc39e1cb9532b7372d47
4d0fc266e6678a8cd771bd2e5fc26aa86fe926fa
84ae47c4eab88ce4deb570edbcd5405e17c1fa77
b2854484a0f6076f623028376c9b0827ba6ee8fe
37b430167f7c8ad0fe1fd900e2dc862aaae0781a
ad52ad06ebedc0eb0ed53b39a934650f4d864f60
618cffc01566fd932adc467e9be2633bd7f3708e
ffc643c4a23d913641669bc078ce407e0dddd01f
86930be3261e454acf01d2667b7272595fee121e
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/TestExecutor.java added or deleted
8305748e531406589d5313ff19cb7032196deba3
dd4f571a76a5680befb0179240cb1ae4d5869dbe
File junit5-engine/src/test/java/org/junit/gen5/engine/junit5/JUnit

File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/TestDescriptorResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/TestDescriptorResolverRegistry.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/TestExecutor.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/UniqueIdTestDescriptorResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/ClassNameTestDescriptorResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/JavaClassTestDescriptor.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/JavaMethodTestDescriptor.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/TestDescriptorResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/TestDescr

File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/execution/injection/SimpleTypeBasedMethodArgumentResolver.java added or deleted
7673bca13658ce50f7592e2586545aa66ad58f29
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/execution/injection/MethodArgumentResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/execution/injection/SingleMethodArgumentResolver.java added or deleted
a665075f0cee8a0c45dceddca340a607db24583a
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/AbstractJavaTestDescriptor.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/AbstractTestDescriptor.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/OldSpecificationResolver.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/descriptor/UniqueId.java added or deleted
File junit5-engine/src/test/java/org/junit/gen5/engi

55e2a148756aeeb55741e7b2e7f4b2e250bdda8d
60eeeee4ba7ad8db3cc23abe3cb4b7af41716fe1
903855d3866abd786a8038f169f837b59feea434
fb9c34e36ab6fd4732c29f2d5af0539a6aa18d3b
87c0ed5afa3f3a6ea6e05849eb124448dcc4f3a7
b5c3ddfb0633075aaef7b00a76631b3b33158ab2
b5c3ddfb0633075aaef7b00a76631b3b33158ab2
40aff3a4037dbf5cab829a4187351d2d1c4909d3
File junit5-api/src/main/java/org/junit/gen5/api/Condition.java added or deleted
File junit5-api/src/main/java/org/junit/gen5/api/Conditional.java added or deleted
File junit5-api/src/main/java/org/junit/gen5/api/Disabled.java added or deleted
File junit5-api/src/main/java/org/junit/gen5/api/support/DisabledCondition.java added or deleted
File junit5-engine/src/main/java/org/junit/gen5/engine/junit5/execution/ConditionalEvaluator.java added or deleted
40aff3a4037dbf5cab829a4187351d2d1c4909d3
File junit5-api/src/main/java/org/junit/gen5/api/Condition.java added or deleted
File junit5-api/src/main/java/org/junit/gen5/api/Conditional.java added or deleted
File junit5

...

File src/main/java/com/p6spy/engine/logging/AbstractP6LogPreparedStatementInvocationHandler.java added or deleted
File src/main/java/com/p6spy/engine/logging/AbstractP6LogStatementInvocationHandler.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogCallableStatementGetResultSetDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogConnectionCommitDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogConnectionRollbackDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogElapsedDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogPreparedStatementAddBatchDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogPreparedStatementExecuteDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogResultSetDelegate.java added or deleted
File src/main/java/com/p6spy/engine/logging/P6LogStatementExecuteBatchDelegate.java added or

21bda23dd2bfd440623573a8515a32be219ff8db
21bda23dd2bfd440623573a8515a32be219ff8db
21bda23dd2bfd440623573a8515a32be219ff8db
21bda23dd2bfd440623573a8515a32be219ff8db
6b8347b2a372732ddf89b87b5d9bd51d9ea1505c
File src/main/java/com/p6spy/engine/common/P6ObjectEqualsDelegate.java added or deleted
File src/main/java/com/p6spy/engine/common/P6ProxyUnwrapDelegate.java added or deleted
File src/main/java/com/p6spy/engine/common/P6WrapperIsWrapperDelegate.java added or deleted
File src/main/java/com/p6spy/engine/common/P6WrapperUnwrapDelegate.java added or deleted
File src/main/java/com/p6spy/engine/event/CompoundJdbcEventListener.java added or deleted
File src/main/java/com/p6spy/engine/event/DefaultEventListener.java added or deleted
File src/main/java/com/p6spy/engine/event/JdbcEventListener.java added or deleted
File src/main/java/com/p6spy/engine/event/SimpleJdbcEventListener.java added or deleted
File src/main/java/com/p6spy/engine/logging/AbstractP6LogPreparedStatementInvocationHandler.ja

File src/main/java/com/p6spy/engine/outage/P6OutageConnectionCommitDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutageConnectionCreateStatementDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutageConnectionInvocationHandler.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutageConnectionPrepareCallDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutageConnectionPrepareStatementDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutageConnectionRollbackDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutagePreparedStatementAddBatchDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutagePreparedStatementExecuteDelegate.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutagePreparedStatementInvocationHandler.java added or deleted
File src/main/java/com/p6spy/engine/outage/P6OutagePreparedSta

File src/main/java/com/p6spy/engine/wrapper/AbstractWrapper.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/CallableStatementWrapper.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/ConnectionWrapper.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/P6Proxy.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/PreparedStatementWrapper.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/ResultSetWrapper.java added or deleted
File src/main/java/com/p6spy/engine/wrapper/StatementWrapper.java added or deleted
File src/test/java/com/p6spy/engine/proxy/GenericInvocationHandlerTest.java added or deleted
File src/test/java/com/p6spy/engine/proxy/MethodNameAndParameterLikeMatcherTest.java added or deleted
File src/test/java/com/p6spy/engine/proxy/MethodNameAndParameterMatcherTest.java added or deleted
File src/test/java/com/p6spy/engine/proxy/MethodNameMatcherTest.java added or deleted
File src/test/java/com/p6spy/engi

c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc4bd9af7d8e36dccef9a1378d
c04c219a50b13fbc